# Face Recognition

---
Autor: Vesna Camile Pivac Alcaraz<br>
Curso: Redes Neuronales Profundas<br>
Programa: Maestria en Ciencia de Datos

## Librerias

In [13]:
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import numpy as np
from typing import Union, AnyStr
import json

In [4]:
print("Tensorflow: ",tf.__version__)
print("Tensorflow-Keras: ",tf.keras.__version__)
print(tf.config.list_physical_devices())
!python --version

Tensorflow:  2.3.0
Tensorflow-Keras:  2.4.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]
Python 3.6.9


## Modelo

In [5]:
json_file = open('./keras-facenet-h5/model.json','r')
loaded_model_json = json_file.read()
json_file.close()

In [6]:
model = model_from_json(loaded_model_json)
model.load_weights('./keras-facenet-h5/model.h5')

In [8]:
FRmodel = model

### Inputs y Outputs del modelo

In [7]:
print(model.inputs)
print(model.outputs)

[<tf.Tensor 'input_1:0' shape=(None, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/batchnorm/add_1:0' shape=(None, 128) dtype=float32>]


## Dataset

In [12]:
def preprocess_image(image: Union[np.ndarray, AnyStr], 
                     shape=(160,160)) -> np.ndarray:
    if isinstance(image, np.ndarray):
        img = tf.keras.preprocessing.image.smart_resize(
            img, shape)
    # If image is path
    else:
        img = tf.keras.preprocessing.image.load_img(
            image, target_size=(160, 160))
        img = np.array(img)
    return img

def img_to_encoding(img: Union[np.ndarray, str], 
                    model) -> np.ndarray:
    if isinstance(img, str):
        img = preprocess_image(img)

    img = np.around(img / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)

    return embedding / np.linalg.norm(embedding, ord=2)

In [14]:
database = {}
database['Vesna'] = img_to_encoding('images/vesna.png', FRmodel).tolist()
database['Estephania'] = img_to_encoding('images/estephania.png', FRmodel).tolist()
database['Danisa'] = img_to_encoding('images/danisa.jpg', FRmodel).tolist()

### Creación del dataset como json

In [15]:
with open('database.json', 'w') as outfile:
    json.dump(database, outfile, indent=4)